In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import math
import tensorflow as tf
from tensorflow.keras.metrics import RootMeanSquaredError
from keras import regularizers
from keras.layers import Dropout, Activation
from keras.callbacks import *
from keras.optimizers import Adam
from datetime import date, datetime,timedelta
# # from google.colab import files
# import io
from keras.models import load_model

Using TensorFlow backend.


In [4]:
def get_news(df2,ticker):

  news_df = pd.read_excel('/Users/pranavsaxena/Desktop/Python/news-data/2020-07-06.xlsx',index_col=0) 
  news_df = news_df[news_df['ticker'] == ticker]
  news_df['neg'][news_df['neu'] == 1] = 0.5  #added
  news_df['pos'][news_df['neu'] == 1] = 0.5  #added
  news_df['neg'] =  news_df.groupby(['date'])['neg'].transform('sum')
  news_df['pos'] =  news_df.groupby(['date'])['pos'].transform('sum')
  news_df = news_df.drop_duplicates(subset=['date'])
  news_df = news_df.rename(columns = {'date':'Date'})
  df2.Date = df2.Date.astype(str)
  news_df.Date = news_df.Date.astype(str)
  # print(news_df[['neg','neu']].head())

  mergedf = pd.merge(df2,news_df,on= 'Date',how='left')
  mergedf['neg'] = mergedf['neg'].fillna(0)
  mergedf['pos'] = mergedf['pos'].fillna(0)
  mergedf = mergedf[['Date','Open','High','Low','Close','Adj Close','Volume','neg','pos']]
  return mergedf

In [5]:
def load_file(stock_name):
    if stock_name == 1:
      df2 = pd.read_csv('/Users/pranavsaxena/Desktop/Python/Dissertation-Datasets/ACN.csv')  
      ticker = 'ACN'  
      news_df = get_news(df2,ticker)
    elif stock_name == 2: 
      df2 = pd.read_csv('/Users/pranavsaxena/Desktop/Python/Dissertation-Datasets/AMZN.csv')  
      ticker = 'AMZN'
      news_df = get_news(df2,ticker)
    elif stock_name == 3:
      df2 = pd.read_csv('/Users/pranavsaxena/Desktop/Python/Dissertation-Datasets/CSCO.csv')  
      ticker = 'CSCO'
      news_df = get_news(df2,ticker)
    elif stock_name == 4:  
      df2 = pd.read_csv('/Users/pranavsaxena/Desktop/Python/Dissertation-Datasets/DELL.csv')  
      ticker = 'DELL'
      news_df = get_news(df2,ticker)
    elif stock_name == 5:  
      df2 = pd.read_csv('/Users/pranavsaxena/Desktop/Python/Dissertation-Datasets/FB.csv')  
      ticker = 'FB'
      news_df = get_news(df2,ticker)
    elif stock_name == 6:  
      df2 = pd.read_csv('/Users/pranavsaxena/Desktop/Python/Dissertation-Datasets/UBER.csv')  
      ticker = 'UBER'
      news_df = get_news(df2,ticker)
    elif stock_name == 7:  
      df2 = pd.read_csv('/Users/pranavsaxena/Desktop/Python/Dissertation-Datasets/VZ.csv')  
      ticker = 'VZ'
      news_df = get_news(df2,ticker)
    
    return (news_df,ticker)

In [6]:
def add_sentiment(news_df):
    news_df['sentiment'] = 1

    for i in range(len(news_df)):
        if(news_df.loc[i,'neg'] > news_df.loc[i,'pos']):
            news_df.loc[i,'sentiment'] = 2
        else:
            news_df.loc[i,'sentiment'] = 1
    df2 = news_df  
    df2['Date'] = pd.to_datetime(df2['Date'])
    df2.set_index('Date',inplace=True)
#     print(df2.tail())
    return df2

In [7]:
def plot_close_price(df2):
    #plotting the data
    plt.figure(figsize=(16,8))
    plt.title('Close Price History')
    plt.plot(df2['Close'][:-1], color='red')
    plt.xlabel('Date', fontsize=18)
    plt.ylabel('Close Price USD', fontsize = 18)
    plt.show()

In [8]:
def scale_data(data):
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data.values.reshape(data.shape[0], 1))
    return data, scaler

In [9]:
def processData(data,lb):
    X,Y = [],[]
    for i in range(len(data)-lb-1):
        X.append(data[i:(i+lb),0])
        Y.append(data[(i+lb+1),0])

    return np.array(X),np.array(Y)

In [10]:
def test_train_split(X,y,train_pct):

    x_train,x_test = X[:int(X.shape[0]*train_pct)],X[int(X.shape[0]*train_pct):]
    y_train,y_test = y[:int(y.shape[0]*train_pct)],y[int(y.shape[0]*train_pct):]

    return (x_train,y_train,x_test,y_test)

In [11]:
def model_load(ticker):
    model1 = load_model("/Users/pranavsaxena/Desktop/Python/saved_model/model_"+ticker+".h5",custom_objects  = {'RootMeanSquaredError': RootMeanSquaredError})
    return model1

In [12]:
def model_evaluate(x_train, y_train,batch_size):
    return model1.evaluate(x_train, y_train, batch_size=batch_size)

In [13]:
def rmse_cal(model,x_test,scaler):
    predictions = model1.predict(x_test)
    predictions = scaler.inverse_transform(predictions)
    y_act = scaler.inverse_transform(y_test.reshape(y_test.shape[0], 1))
    #get the root mean square error(RMSE)
    rmse = np.sqrt(np.mean(predictions - y_act)**2)
    return rmse

In [14]:
def plot_model(trained_model, train, test, actual):

    x_train_prediction = trained_model.predict(train)
    x_test_prediction = trained_model.predict(test)
  
#     plt.figure(figsize=(20,10))

#     plt.plot(scaler.inverse_transform(actual.reshape(-1,1)), label="Actual")
#     plt.plot(scaler.inverse_transform(x_train_prediction), label="Train", alpha=0.8)
#     plt.plot(np.linspace(x_train.shape[0]+1,x_train.shape[0]+1 + len(x_test_prediction), len(x_test_prediction)), scaler.inverse_transform(x_test_prediction), label="Test", color='orange', alpha=0.8)
#     plt.legend(fontsize="xx-large")
    return (scaler.inverse_transform(x_train_prediction),scaler.inverse_transform(x_test_prediction))

In [15]:
def display_predict_data(sclaer,test_pred,y):
    df = scaler.inverse_transform(y[int(y.shape[0]*0.8):].reshape(-1,1))
    df = pd.DataFrame(df)
    df.columns=['Close']

    df['Pred'] = test_pred
    df['Difference'] = df['Close'] - df['Pred']

    return df

In [16]:
def get_min(df):
  return dataset['Close'][:training_size][-1]

In [17]:
def get_latest_stock_price(test_pred):
    return str(test_pred[-1])

In [18]:
def get_pred_price(df):
  pred_list = []
  for item in df['Pred']:
    pred_list.append(item)
  return pred_list

def get_close_price(df):
  pred_list = []
  for item in df['Close']:
    pred_list.append(item)
  return pred_list  

In [19]:
def get_diff_pct(pred_price,close_min):
    return ((pred_price - close_min)/close_min) * 100

In [20]:
def get_stock(ticker,stocks):
    if ticker == 'ACN':
        return stocks[ticker]
    elif ticker == 'AMZN':
        return stocks[ticker]  
    elif ticker == 'CSCO':
        return stocks[ticker]  
    elif ticker == 'DELL':
        return stocks[ticker]  
    elif ticker == 'FB':
        return stocks[ticker]  
    elif ticker == 'UBER':
        return stocks[ticker]  
    elif ticker == 'VZ':
        return stocks[ticker]  

In [21]:
def update_stock(ticker,stocks,stock_count,action):
    if action == 'sell':
        stocks[ticker] = stocks[ticker] - stock_count
    elif action == 'buy':  
        stocks[ticker] = stocks[ticker] + stock_count

In [22]:
def get_news_sentiment(df2):
    sentiment = df2['sentiment'][-1]
    # sentiment = 2
    return sentiment

In [23]:
def transaction(close_min,stock_in_hand,ticker,pred_price,stocks,open_cash,open_price,hyp_parm,df2):
    
    count_sell = 0
    count_buy  = 0
    count_hold = 0
    close_diff_pct = (get_diff_pct(pred_price,close_min))   
    news_sentiment = get_news_sentiment(df2)
    balance_low_ind = False
    
    #SELL CONDITIONS CHECK
#     if np.abs(pred_price - close_min) < 100: # the model might have predicted the worng value. The fluctuation is too much.
    
    if close_diff_pct >= 3.0 and close_diff_pct > 0:   # the difference in the predicted stock price and last bought value is greater than 3%

        if stock_in_hand != 0:  # we do have some stocks in hand to do the transaction

            if close_diff_pct >= 5.0:  # if the percent difference is greater than 5% then sell all the stocks expect last 5 stcoks

                if stock_in_hand > 5: # sell all the stocks expect 5(let 5 stocks be in balance) 
                    action = 'sell'
                    open_cash = open_cash + hyp_parm * pred_price 
                    stock_count = hyp_parm
                    update_stock(ticker,stocks,stock_count,action)
                    count_sell += stock_count
                    return(close_min,action,open_cash,stocks[ticker],balance_low_ind,count_sell)

                elif stock_in_hand > 3: # save atleast 3 stocks in bucket
                    action = 'sell'
                    open_cash = open_cash + pred_price # sell only one stock
                    stock_count = 1 
                    update_stock(ticker,stocks,stock_count,action)
                    count_sell += stock_count
                    return(close_min,action,open_cash,stocks[ticker],balance_low_ind,count_sell)

                else: # cannot perform transaction due to insufficient Stocks
                    balance_low_ind = True
#                         print("Insufficient Stocks in hand: ",stocks[ticker]) 
#                         print()
                    action = 'Can not perform the transaction due to insufficient Stocks' 
                    balance_low_ind = True
                    return(close_min,action,open_cash,stocks[ticker],balance_low_ind,count_sell)

            else: # sell only one stock as percent difference is > 3% but less than 5% and sentiment is negative

                if(news_sentiment == 2): # news sentiment is negative, so it might fall tomorrow
                    action = 'sell'
                    open_cash = open_cash + pred_price 
                    stock_count = 1 
                    update_stock(ticker,stocks,stock_count,action)
                    count_sell += stock_count
                    return(close_min,action,open_cash,stocks[ticker],balance_low_ind,count_sell)

                else:
                    action = 'hold'    # hold the stock as it might rise more due to positive news sentiment
                    count_hold += 1
                    return(close_min,action,open_cash,stocks[ticker],balance_low_ind,count_hold)

        else:  # cannot perform transaction due to insufficient Stocks

            balance_low_ind = True
#                 print("Insufficient Stocks in hand: ",stocks[ticker]) 
#                 print()
            action = 'Can not perform the transaction due to insufficient Stocks' 
            balance_low_ind = True
            return(close_min,action,open_cash,stocks[ticker],balance_low_ind,count_hold)

    # BUY CONDITIONS CHECK

    elif (close_diff_pct) <= -3.0 and close_diff_pct < 0: # the difference in the predicted stock price and last bought value is less than 3%

        if (close_diff_pct) <= -5.0: # if the percent difference is less than 5% then sell all the stocks expect last 5 stocks

            if(open_cash - pred_price > 2000): # it checks if we have open cash still greater than 2000 after buying one stock

                if ((hyp_parm * pred_price <= open_cash) and (open_cash - (hyp_parm * pred_price) >= 2000)) : # buy 5 stocks if we still have 2000 euros in balance after buying 5 stocks

                    action = 'buy'
                    open_cash = open_cash - (hyp_parm * pred_price) # buy 5 stocks
                    stock_count = hyp_parm
                    close_min = pred_price
                    update_stock(ticker,stocks,stock_count,action)
                    count_buy += stock_count
                    return(close_min,action,open_cash,stocks[ticker],balance_low_ind,count_buy)

                else:  # if balance is still greater than 2000 after buying one stock

                    action = 'buy'
                    open_cash = open_cash - pred_price # buy 1 stock
                    stock_count = 1
                    close_min = pred_price
                    update_stock(ticker,stocks,stock_count,action)
                    count_buy += stock_count
                    return(close_min,action,open_cash,stocks[ticker],balance_low_ind,count_buy)
            else:
                action = 'hold'    
                return(close_min,action,open_cash,stocks[ticker],balance_low_ind,count_hold)

        elif(open_cash - pred_price > 2000): # it checks if we have open cash still greater than 2000 after buying one stock

            if(news_sentiment == 1): # due to postive news sentiment, it might increase from tomorrow
                action = 'buy'
                open_cash = open_cash - pred_price # buy 1 stock
                stock_count = 1
                close_min = pred_price
                update_stock(ticker,stocks,stock_count,action)
                count_buy += stock_count
                return(close_min,action,open_cash,stocks[ticker],balance_low_ind,count_buy)
            else:
                action = 'hold'    # hold the stock as it might fall more due to negative news sentiment
                count_hold += 1
                return(close_min,action,open_cash,stocks[ticker],balance_low_ind,count_hold)


        else:
#                 print("Insufficient Balance") 
#                 print()
            action = 'Can not perform the transaction due to the balance less than 2000' 
            balance_low_ind = True
            return(close_min,action,open_cash,stocks[ticker],balance_low_ind,count_hold)

    # HOLD CONDITIONS CHECK

    else:
        action = 'hold'    
        count_hold += 1
        return(close_min,action,open_cash,stocks[ticker],balance_low_ind,count_hold)
    
#     else:
# #         print('Predicted price varies a lot from the last closed price')
#         action = 'hold'
#         count_hold += 1
#         return(close_min,action,open_cash,stocks[ticker],balance_low_ind,count_hold)
    

In [24]:
def get_stock_data(ticker,df,latest_stock_price):
    close_min = get_min(df)
    stock_buy_price[ticker] = close_min
    pred_price = get_pred_price(df)
    transaction_count = 0
    latest_stock_price[ticker] = get_latest_stock_price(test_pred)
    
    return(pred_price,latest_stock_price)

In [25]:
def cal_initial_value(stocks,stock_buy_price,open_cash):
    total=0
    for key in stocks:   
        total=total + stocks[key] * stock_buy_price[key]
    total = total + open_cash
    print("Total initial portfolio value is :" ,round(total,3)) 
    return total

In [26]:
def stock_trading(pred_price,ticker,stock_pred_price,close_min,stocks,open_cash,hyp_parm,df2):
    
    transaction_count = 0
    balance_low_ind = False
    count_sell = 0
    count_buy  = 0
    count_hold = 0
    
    for new_price in pred_price:

        stock_pred_price[ticker] = new_price
        stock_in_hand = get_stock(ticker,stocks)

        close_min,action_taken, open_cash, stocks_left,balance_low_ind,count = transaction(close_min,stock_in_hand,ticker,new_price,stocks,open_cash,balance_low_ind,hyp_parm,df2)

        transaction_count = transaction_count + 1
        if action_taken == 'sell':
            stock_buy_price[ticker] = new_price
            close_min = new_price
            count_sell += count
        elif action_taken == 'buy':
            stock_buy_price[ticker] = new_price
            close_min = new_price
            count_buy += count
        else:
            count_hold += count

        if balance_low_ind == True or stocks_left == 0:
            break  
    
    print("Stock traded is : ",ticker)
    print("Transaction number: ",transaction_count) 
    print("Number of times stocks sold: ",count_sell)
    print("Number of times stocks bought: ",count_buy)
    print("Number of times stocks hold: ",count_hold) 

    return open_cash


In [27]:
def get_overall_value(open_cash,stocks,stock_pred_price):
    total=0
    for key in stocks:   
        total=total + stocks[key] * stock_pred_price[key]
    total = total + open_cash
    print("Total portfolio value is :" ,round(total,3)) 
    print()
    return total

In [28]:
def init_prices():
    stocks = {'ACN':50,'AMZN':50,'CSCO':50,'DELL':50,'FB':50,'UBER':50,'VZ':50}
    stock_buy_price = {'ACN':1,'AMZN':1,'CSCO':1,'DELL':1,'FB':1,'UBER':1,'VZ':1}
    stock_pred_price = {'ACN':0,'AMZN':0,'CSCO':0,'DELL':0,'FB':0,'UBER':0,'VZ':0}
    latest_stock_price = {'ACN':0,'AMZN':0,'CSCO':0,'DELL':0,'FB':0,'UBER':0,'VZ':0}
    open_cash = 1000000 
    return (stocks,stock_buy_price,stock_pred_price,latest_stock_price,open_cash)

In [32]:
for stock_name in range(1,8):
    
    news_df,ticker = load_file(stock_name)
    
    df2 = add_sentiment(news_df)
    
    # create a new data frame with only 'Close column'
    data = df2.filter(['Close'])
    dataset = data
    data,scaler = scale_data(data)
    
    look_back = 6
    X,y = processData(data,look_back)
    
    #Reshape data into (Sample,Timestep,Features) 
    x_train,y_train,x_test,y_test = test_train_split(X,y,0.80)
    x_train = x_train.reshape((x_train.shape[0],x_train.shape[1],1))
    x_test = x_test.reshape((x_test.shape[0],x_test.shape[1],1))

    training_size=int(len(data)*0.80)
    
    model1 = model_load(ticker)
    
    train_pred,test_pred=plot_model(model1, x_train, x_test, y)

    df = display_predict_data(scaler,test_pred,y)
    
    pred_price,latest_stock_price = get_stock_data(ticker,df,latest_stock_price)
    best = 0
    param = 0

    for hyp_param in range(10,15):
        stocks,stock_buy_price,stock_pred_price,latest_stock_price,open_cash = init_prices()
        total_old = cal_initial_value(stocks,stock_buy_price,open_cash)
        open_cash = stock_trading(pred_price,ticker,stock_pred_price,stock_buy_price[ticker],stocks,open_cash,hyp_param,df2)
    
        total_new = get_overall_value(open_cash,stocks,stock_buy_price)
        if total_new > best:
            best = total_new
            param = hyp_param
            
    print()
    print("Best parameter is: ",param)
    print()    


Total initial portfolio value is : 1000350
Stock traded is :  ACN
Transaction number:  242
Number of times stocks sold:  50
Number of times stocks bought:  3
Number of times stocks hold:  233
Total portfolio value is : 1006480.73

Total initial portfolio value is : 1000350
Stock traded is :  ACN
Transaction number:  242
Number of times stocks sold:  55
Number of times stocks bought:  3
Number of times stocks hold:  233
Total portfolio value is : 1006448.824

Total initial portfolio value is : 1000350
Stock traded is :  ACN
Transaction number:  275
Number of times stocks sold:  50
Number of times stocks bought:  3
Number of times stocks hold:  265
Total portfolio value is : 1006443.378

Total initial portfolio value is : 1000350
Stock traded is :  ACN
Transaction number:  207
Number of times stocks sold:  52
Number of times stocks bought:  3
Number of times stocks hold:  199
Total portfolio value is : 1006378.825

Total initial portfolio value is : 1000350
Stock traded is :  ACN
Transac

In [30]:
print(stock_buy_price)
print()
print(stock_pred_price)
print()
print(latest_stock_price)

{'ACN': 1, 'AMZN': 1, 'CSCO': 1, 'DELL': 1, 'FB': 1, 'UBER': 1, 'VZ': 47.21974563598633}

{'ACN': 0, 'AMZN': 0, 'CSCO': 0, 'DELL': 0, 'FB': 0, 'UBER': 0, 'VZ': 49.739620208740234}

{'ACN': 0, 'AMZN': 0, 'CSCO': 0, 'DELL': 0, 'FB': 0, 'UBER': 0, 'VZ': 0}
